Importing needed Libs, functions / Creating a DB File

In [1]:
import sqlite3
import pandas as pd
from random import randint
from datetime import timedelta, datetime

conn = sqlite3.connect("tables.db")
cur = conn.cursor()

Functions, Important variables Initialization

In [2]:
#Initializing a randbetween dates function
def random_datetime(start_date, end_date):
    delta = end_date - start_date
    return start_date + timedelta(randint(0, delta.days), randint(0, delta.seconds))


#Function to load SQL Queries into a dictionary
def SQL_to_dict(dictionary: dict, list_of_sql_tables: sqlite3.Cursor, key_index_column: int):

    if len(dictionary) != len(list_of_sql_tables):
        raise IndexError("dict length does not match the number of sql_tables")

    keys = list(dictionary)
    for i in range(len(keys)):
        dictionary[keys[i]] = [j[key_index_column] for j in list_of_sql_tables[i].fetchall()]

    return dictionary

start_date = datetime.strptime('1/1/2021 12:00 AM', '%m/%d/%Y %I:%M %p')
end_date = datetime.strptime('12/31/2021 11:59 PM', '%m/%d/%Y %I:%M %p')
area_records = 27
restaurant_records = 100
user_records = 20000
order_records = 500000

Area Table Initialization (27 Egyptian governorate)

In [3]:
#Download from https://github.com/Tech-Labs/egypt-governorates-and-cities-db
governorate = pd.read_csv("governorates.csv")
governorate = governorate[["id", "governorate_name_en"]].rename(columns={"governorate_name_en":"name"})

governorate.to_sql(name="AREAS", 
    con=conn, 
    if_exists="replace", 
    index=False, 
    dtype={"id": "INTEGER PRIMARY KEY",
        "name": "TEXT NOT NULL"})

Restaurants Table Initialization (100 Restuarant)

In [4]:
restaurants = pd.DataFrame({"id": [i+1 for i in range(restaurant_records)], 
    "name": ["Restaurant"+(len(str(restaurant_records))-len(str(i+1)))*'0' + str(i+1) for i in range(restaurant_records)], 
    "area_id": [randint(1, area_records) for i in range(restaurant_records)]})

restaurants.to_sql(name="RESTAURANTS", 
    con=conn, 
    if_exists="replace", 
    index=False, 
    dtype={"id": "INTEGER PRIMARY KEY", 
        "name": "TEXT NOT NULL", 
        "area_id": "INTEGER NOT NULL"})

User Table Initialization (20k record)

In [5]:
user = pd.DataFrame({"id": [i+1 for i in range(user_records)], 
    "name": ["User"+(len(str(user_records))-len(str(i+1)))*'0' + str(i+1) for i in range(user_records)], 
    "email": ["User_"+str(i+1)+"@Domain.com" for i in range(user_records)]})

user.to_sql(name="USERS", 
    con=conn, 
    if_exists="replace", 
    index=False, 
    dtype={"id": "INTEGER PRIMARY KEY", 
    "name": "TEXT NOT NULL", 
    "email": "TEXT NOT NULL"})

Order Table Initialization (500K record, date range is in one year)

In [6]:
order = pd.DataFrame({"id": [(i+1) for i in range(order_records)], 
    "short_code": [((len(str(order_records))-len(str(i+1)))*'0' + str(i+1)) for i in range(order_records)], 
    "timestamp": [random_datetime(start_date, end_date) for i in range(order_records)], 
    "order_value": [float(randint(1, 1000)) for i in range(order_records)], 
    "discount": [randint(1, 17)/20 if i%5 == 0 else 0 for i in range(order_records)], #Discount is Multiple of 5% (Maximum 85%), 1 Discount for each 5 orders
    "user_id": [randint(1, user_records) for i in range(order_records)], 
    "restaurant_id": [randint(1, restaurant_records) for i in range(order_records)], 
    "promo_name": ["Promo" + str(randint(1, 20)) for i in range(order_records)]})

#Apply the relationship between discount & order value
order["order_value"] = order["order_value"] - (order["order_value"] * order["discount"])

#Renaming the promo_name of the records with no discount
order.loc[order["discount"] == 0, "promo_name"] = "N/A"

order.to_sql(name="ORDER_TABLE", 
    con=conn, 
    if_exists="replace", 
    index=False, 
    dtype={"id": "INTEGER PRIMARY KEY", 
    "short_code": "TEXT NOT NULL", 
    "timestamp": "TEXT NOT NULL", 
    "order_value": "FLOAT NOT NULL", 
    "discount": "FLOAT NOT NULL", 
    "user_id": "INTEGER NOT NULL", 
    "restaurant_id": "INTEGER NOT NULL", 
    "promo_name": "TEXT NOT NULL"})

Sales per area Queries

In [ ]:
#Orders per area
orders_number_per_area = conn.execute("SELECT \
        AREAS.name AS Area, \
        COUNT(*) AS Orders \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        INNER JOIN AREAS \
            ON RESTAURANTS.area_id = AREAS.id \
        GROUP BY Area \
        ORDER BY Area")

#Unique users per area
unique_users_per_area = conn.execute("SELECT \
        AREAS.name AS Area, \
        COUNT(DISTINCT ORDER_TABLE.user_id) AS Users \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        INNER JOIN AREAS \
            ON RESTAURANTS.area_id = AREAS.id \
        GROUP BY Area \
        ORDER BY Area")

#Number of new users per area
new_users_per_area = conn.execute("WITH min_purchase_per_user AS \
        (SELECT \
            AREAS.name AS Area, \
            MIN(ORDER_TABLE.timestamp) AS min_date, \
            ORDER_TABLE.user_id AS user_id \
        FROM ORDER_TABLE \
            INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
            INNER JOIN AREAS \
                ON RESTAURANTS.area_id = AREAS.id \
            GROUP BY user_id) \
    SELECT \
        Area, \
        Count(*) AS New_users \
    FROM min_purchase_per_user \
        GROUP BY Area \
        ORDER BY Area")

#Number of returning users
returning_users_per_area = conn.execute("WITH ORDER_PER_AREA AS \
        (SELECT \
            ORDER_TABLE.user_id AS User, \
            AREAS.name As Area, \
            COUNT(*) As User_Count \
        FROM ORDER_TABLE \
            INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
            INNER JOIN AREAS \
                ON RESTAURANTS.area_id = AREAS.id \
            GROUP BY Area, User) \
    SELECT \
        Area, \
        COUNT(Area) AS Returning_users\
    FROM ORDER_PER_AREA \
        WHERE User_count > 1 \
        GROUP BY Area \
        ORDER BY Area")

#Organic orders per area
organic_orders_per_area = conn.execute("SELECT \
        AREAS.name AS Area, \
        COUNT(ORDER_TABLE.discount) AS Discount \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        INNER JOIN AREAS \
            ON RESTAURANTS.area_id = AREAS.id \
        WHERE Discount = 0 \
    GROUP BY Area \
    ORDER BY Area")

#Orders value per area, Note: This is the value after discount
orders_value_per_area = conn.execute("SELECT \
        AREAS.name AS Area, \
        SUM(ORDER_TABLE.order_value) AS Value \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        INNER JOIN AREAS \
            ON RESTAURANTS.area_id = AREAS.id \
        GROUP BY Area \
        ORDER BY Area")

#Discount value per area
#Discount_value = Actual_value - (Order_Value: AKA AFTER The discount)
#Discount_value = X - (X * Discount_persent) = X (1 - Discount_persent) <==> X = Discount_value/(1 - Discount_persent) AS X = Actual_value
discount_value_per_area = conn.execute("SELECT \
        AREAS.name AS Area, \
        SUM((ORDER_TABLE.order_value/(1 - ORDER_TABLE.discount)) - ORDER_TABLE.order_value) AS Discount_value \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        INNER JOIN AREAS \
            ON RESTAURANTS.area_id = AREAS.id \
        GROUP BY Area \
        ORDER BY Area")

#Discount percentage per area
discount_percentage_per_area = conn.execute("WITH DISCOUNT_PERCENTAGE_PER_AREA AS \
        (SELECT \
            AREAS.name AS Area, \
            SUM(ORDER_TABLE.order_value/(1 - ORDER_TABLE.discount)) AS Actual_value, \
            SUM(ORDER_TABLE.order_value) AS Value \
        FROM ORDER_TABLE \
            INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
            INNER JOIN AREAS \
                ON RESTAURANTS.area_id = AREAS.id \
            GROUP BY Area) \
    SELECT \
        Area, \
        (1 - (Value / Actual_value)) AS discount_percentage \
    FROM DISCOUNT_PERCENTAGE_PER_AREA \
        ORDER BY Area")

#This is a helper Query
area_names = conn.execute("SELECT \
        COUNT(*) AS Anything, \
        AREAS.name AS Area \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        INNER JOIN AREAS \
            ON RESTAURANTS.area_id = AREAS.id \
        GROUP BY Area \
        ORDER BY Area")

Loading Sales per Area Data into a pandas DataFrame

In [ ]:
Area = {"Area": [], 
    "Orders": [], 
    "Unique_users": [], 
    "New_users": [], 
    "Returning_users": [], 
    "Organic_orders": [], 
    "Orders_value": [], 
    "Discount_value": [], 
    "Discount_percentage": []}

list_of_area_tables = [area_names, 
    orders_number_per_area, 
    unique_users_per_area, 
    new_users_per_area, 
    returning_users_per_area, 
    organic_orders_per_area, 
    orders_value_per_area, 
    discount_value_per_area, 
    discount_percentage_per_area]

Area = SQL_to_dict(dictionary=Area, 
    list_of_sql_tables=list_of_area_tables, 
    key_index_column= 1)

Area_Analysis = pd.DataFrame(Area)

Area_Analysis

Monthly Sales

In [ ]:
#Monthly cohort
#Customized to count zero for Month's without new users
monthly_cohort = conn.execute("WITH MIN_MONTH AS \
        (SELECT \
            id AS id, \
            MIN(STRFTIME('%m/%Y', ORDER_TABLE.timestamp)) AS Min_month, \
            ORDER_TABLE.user_id AS user_id \
        FROM ORDER_TABLE \
            GROUP BY user_id) \
    SELECT \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) Month, \
        COUNT(MIN_MONTH.user_id) AS New_users \
    FROM ORDER_TABLE\
        LEFT JOIN MIN_MONTH \
            ON (MIN_MONTH.id = ORDER_TABLE.id) \
        GROUP BY Month \
        ORDER BY Month")

#Monthly orders
monthly_orders = conn.execute("SELECT \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
        COUNT(*) \
    FROM ORDER_TABLE \
        GROUP BY Month \
        ORDER BY Month")

#Organic orders monthly
monthly_organic_orders = conn.execute("SELECT \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
        COUNT(*) AS Organic_orders \
    FROM ORDER_TABLE \
        WHERE ORDER_TABLE.discount = 0 \
        GROUP BY Month \
        ORDER BY Month")

#Monthly discount percentage
monthly_discount_percentage = conn.execute("WITH MONTHLY_ORDERS_VALUE AS \
        (SELECT \
            STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
            SUM(ORDER_TABLE.order_value/(1 - ORDER_TABLE.discount)) AS Actual_value, \
            SUM(ORDER_TABLE.order_value) AS Value_after_discount  \
        FROM ORDER_TABLE \
            GROUP BY Month) \
    SELECT \
        Month, \
        (1 - (Value_after_discount / Actual_value)) AS discount_percentage \
    FROM MONTHLY_ORDERS_VALUE \
        ORDER BY Month")

#This is a helper Query
month_names = conn.execute("SELECT \
        COUNT(*) AS ANYTHING, \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month \
    FROM ORDER_TABLE \
        GROUP BY Month \
        ORDER BY Month")
#Using COUNT(*) To Fill the first column in the data frame..

Loading Monthly sales Data into a pandas DataFrame

In [ ]:
Monthly = {"Month": [], 
    "New_Users": [], 
    "Orders": [], 
    "Organic_orders": [], 
    "Discount_percentage": []}

list_of_monthly_tables = [month_names, 
    monthly_cohort, 
    monthly_orders, 
    monthly_organic_orders, 
    monthly_discount_percentage]

Monthly = SQL_to_dict(dictionary=Monthly, 
    list_of_sql_tables=list_of_monthly_tables, 
    key_index_column= 1)

Monthly_Analysis = pd.DataFrame(Monthly)

Monthly_Analysis

Monthly Queries per restaurant

In [ ]:
#Monthly orders per restaurant
monthly_orders_per_restaurant = conn.execute("SELECT \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
        RESTAURANTS.name AS Restaurant, \
        COUNT(*) AS Orders \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        GROUP BY Month, Restaurant \
        ORDER BY Month, Restaurant")

#Monthly unique users per restaurant
monthly_unique_users_per_restaurant = conn.execute("WITH MONTHLY_UNIQUE_USERS AS \
        (SELECT \
            STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
            RESTAURANTS.name AS Restaurant, \
            ORDER_TABLE.user_id AS User \
        FROM ORDER_TABLE \
            INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
            GROUP BY Month, Restaurant, User) \
    SELECT \
        Month, \
        Restaurant, \
        COUNT(*) AS Unique_users \
    FROM MONTHLY_UNIQUE_USERS \
        GROUP BY Month, Restaurant \
        ORDER BY Month, Restaurant")

#Monthly new users per restaurant
monthly_new_users_per_restaurant = conn.execute("WITH MONTHLY_NEW_USERS AS \
        (SELECT \
            ORDER_TABLE.id as id, \
            MIN(ORDER_TABLE.timestamp) AS Min_date, \
            RESTAURANTS.name AS Restaurant_name, \
            ORDER_TABLE.user_id AS User \
        FROM ORDER_TABLE \
            INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
            GROUP BY User) \
    SELECT \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
        RESTAURANTS.name AS Restaurant, \
        COUNT(MONTHLY_NEW_USERS.User) AS New_users \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        LEFT JOIN MONTHLY_NEW_USERS \
                ON ORDER_TABLE.id = MONTHLY_NEW_USERS.id \
        GROUP BY Month, Restaurant \
        ORDER BY Month, Restaurant")

#monthly organic orders percentage per restaurant
monthly_organic_orders_percentage_per_restaurant = conn.execute("SELECT \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
        RESTAURANTS.name AS Restaurant, \
        CAST(SUM(CASE WHEN ORDER_TABLE.discount = 0 THEN 1 ELSE 0 END) AS REAL)/CAST(COUNT(*)  AS REAL)  AS Organic_percentage\
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
                ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        GROUP BY Month, Restaurant \
        ORDER BY Month, Restaurant")

#This is a helper Query
for_month_name = conn.execute("SELECT \
        COUNT(*) AS Anything, \
        RESTAURANTS.name AS Restaurant, \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        GROUP BY Month, Restaurant \
        ORDER BY Month, Restaurant")

for_restaurant_name = conn.execute("SELECT \
        COUNT(*) AS Anything, \
        STRFTIME('%m/%Y', ORDER_TABLE.timestamp) AS Month, \
        RESTAURANTS.name AS Restaurant \
    FROM ORDER_TABLE \
        INNER JOIN RESTAURANTS \
            ON ORDER_TABLE.restaurant_id = RESTAURANTS.id \
        GROUP BY Month, Restaurant \
        ORDER BY Month, Restaurant")

Loading Monthly per resturant sales Data into a pandas DataFrame

In [ ]:
Monthly_per_restaurant = {"Month": [], 
    "Restaurant_name": [], 
    "Orders": [], 
    "Unique_users": [], 
    "New_users": [],
    "Organic_orders_percentage": []}

list_of_monthly_per_restaurant_tables = [for_month_name, 
    for_restaurant_name, 
    monthly_orders_per_restaurant, 
    monthly_unique_users_per_restaurant, 
    monthly_new_users_per_restaurant, 
    monthly_organic_orders_percentage_per_restaurant]

Monthly_per_restaurant = SQL_to_dict(dictionary=Monthly_per_restaurant, 
    list_of_sql_tables=list_of_monthly_per_restaurant_tables, 
    key_index_column= 2)

Monthly_restaurant_Analysis = pd.DataFrame(Monthly_per_restaurant)

Monthly_restaurant_Analysis

Loading data into excel file

In [10]:
with pd.ExcelWriter("Restaurant_data.xlsx") as writer:

    Area_Analysis.to_excel(excel_writer= writer, sheet_name='Area_Data', index=False)
    Monthly_Analysis.to_excel(excel_writer= writer, sheet_name='Monthly_Data', index=False)
    Monthly_restaurant_Analysis.to_excel(excel_writer= writer, sheet_name='Monthly_Restaurants_Data', index=False)